In [1]:
import numpy as np
import pandas as pd
import math
import os
import swifter
from names_dataset import NameDataset
import functools
import operator
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
from string_grouper import match_strings

C:\Users\fjying\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\input_2012")

In [3]:
#import the dataset
patents = pd.read_csv("ad20121231-02.csv")

C:\Users\fjying\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (19,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,88,89,90,91,92,94,95,96,97,98,100,101,102,103,104,106,107,108,109,110,112,113,114,115,116,118,119,120,121,122,124,125,126,127,128,130,131,132,133,134,135,136,137,138,139,140,142,143,144,145,146,148,149,150,151,152,154,155,156,157,158,159,160,161,162,163,164,166,167,168,169,170,172,173,174,175,176,178,179,180,181,182,184,185,186,187,188,189) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#preprocess company name
#Convert company names to standardized name for better matching

In [5]:
def name_preprocess(x):
    if type(x) != str:
        return x
    x = x.lower().replace(",", "").replace("-", "").replace("*", "").replace("!", "").replace("(", "").replace(")", "")
    x = x.replace("'", "").replace(";", "").replace("/", "").replace("-", "").replace(r"\(.*\)", "").replace("u.s", "us")
    x = x.replace("u s", "us").replace(".", "").strip()
    return x

In [6]:
#drop endwords in the company name; remove influence of these words on matching
word_lists = ["inc", "company", "companies", "corporation", "corporations", "com", "co", "ltd", "co ltd", "corp", "gmbh", "group", 
              "gp", "plc",  "cl a", "cl b", "cl i", "-redh", "/de", "/md", "&", "lp", "llc", "limited liability",
              "svc", "holdings", "international", "incorporated", 'financial', "technology", "technologies", 
              "limited", "cdti", "sa", "bv", "sl",  "a/s", "the", "gmbh", "nv", "ag", "kg", "holding", 'association', 'bank', 'corporate', 
                          'council','enterprise','foundation','group','inc','incorporated','institute',  'll, ''llc', 'llp', 'ltd', 'management', 'organization', 
                          'partners', 'pllc', 'services', 'utitlies']

#drop end words which do not have spaces between words
def dropendwords(x, word_lists):
    if type(x) != str:
        return x
    for word in word_lists:
        length = len(word)
        if x[-length:] == word:
            return x[0:-length].strip()
    return x


In [7]:
#sort patents by patent ids and last update date
patents = patents.sort_values(by =['patent_ids', 'recorded_date'])

In [8]:
patents = patents.dropna(subset  = ['patent_ids', 'assignor', 'name_0', 'recorded_date'])

In [9]:
#drop duplicated entry
patents = patents.drop_duplicates( ['patent_ids', 'assignor', 'name_0', 'recorded_date'], keep = 'first')

In [10]:
names = patents.head(1).sort_index(axis = 1).columns

In [11]:
names = [s for s in names if s[0:4] =='name']
names.append('assignor')

In [12]:
for name in names:
    patents[name] = patents[name].swifter.apply(lambda x: name_preprocess(x)).swifter.apply(lambda x: dropendwords(x, word_lists)).swifter.apply(lambda x: dropendwords(x, word_lists))

In [13]:
#identify patent transfer by fuzzy match assignor names/address with assignee names/address

In [14]:
#Import the dataset of names
m = NameDataset()

In [15]:
def identify_name(x):
    if m.search_last_name(x) or m.search_first_name(x):
        return True
    else:
        return False

In [16]:
def identify_person(name):
    temp = name.split(' ')
    if len(temp[-1]) == 1 and temp[-1].isalpha() and len(temp) == 3 and sum([identify_name(name) for name in temp[:-1]]) == 2:
        return True
    elif sum([identify_name(name) for name in temp]) == len(temp) and len(temp)>=2:
        return True
    return False

In [17]:
def identify_notmeaningful_one(temp):
    if identify_person(temp['assignor']):
        return True
    match = False
    for i in range(temp['total_number_assignees']):
        if (fuzz.token_sort_ratio(temp['assignor'],temp['name_' + str(i)]) >= 80):
            match = True
            break
        if set(temp['assignor'].split()) & set(temp['name_' + str(i)].split()):
            match = True
            break
    return match

In [18]:
def identify_notmeaningful_one_sub(temp):
    match = False
    for i in range(temp['total_number_assignees']):
        if (fuzz.token_sort_ratio(temp['assignor'],temp['name_' + str(i)]) >= 80):
            match = True
            break
        if set(temp['assignor'].split()) & set(temp['name_' + str(i)].split()):
            match = True
            break
    return match

In [19]:
#Process each patent id
def identify_patent_not_meaningful(temp):
    notmeaningful = []
    addresses = ['country-name','state', 'city', 'postcode','address-2', 'address-1']
    #score threshold is the simialirty threshold
    for i, assignment in temp.iterrows():
        assignor  = assignment['assignor']
        if identify_person(assignor):
            notmeaningful.append(True)
            continue
        try:
            idx = list(temp['name_0']).index(assignor)
            assignor_ad = temp.iloc[idx]
            for i in range(assignment['total_number_assignees']):
                for address in addresses:
                    if address + '_0' in assignor_ad.keys() and address + '_' + str(i) in assignment.keys():
                        score = fuzz.token_sort_ratio(assignor_ad[address + '_0' ], assignment[address + '_' + str(i)])
                        if score < 85:
                            break       
                if score >= 85:
                    notmeaningful.append(True)
                    break
            if score < 85:
                notmeaningful.append(False)
        except ValueError:
            match = identify_notmeaningful_one_sub(assignment)
            notmeaningful.append(match)
    return notmeaningful
            

In [20]:
def final_identify_not_meaningful_single(patentid):  
    temp = patents[patents['patent_ids'] == patentid]
    #drop empty columns
    temp = temp.dropna(how = 'all', axis = 1)
    notmeaningful = []
    match0 = identify_notmeaningful_one(temp.iloc[0])    
    notmeaningful.append(match0)
    return notmeaningful

In [21]:
def final_identify_not_meaningful_multiple(patentid): 
    temp = patents[patents['patent_ids'] == patentid]
    #drop empty columns
    temp = temp.dropna(how = 'all', axis = 1)
    notmeaningful = []
    match1 = identify_patent_not_meaningful(temp)
    notmeaningful.extend(match1)
    return notmeaningful

In [22]:
patentids_multiple_entries = np.unique(patents['patent_ids'][patents['patent_ids'].duplicated()].values)

In [23]:
patents_multiple_entries = patents[patents['patent_ids'].isin(patentids_multiple_entries)]
patents_single_entry = patents[~patents['patent_ids'].isin(patentids_multiple_entries)]

In [24]:
patentids = np.unique(patents_single_entry['patent_ids'].values)

In [25]:
with Parallel(n_jobs = -1, verbose = 1, prefer = 'threads') as parallel:
    results = parallel(delayed(final_identify_not_meaningful_single)(patentid) for patentid in patentids)    

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:   58.9s
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 11242 tasks  

In [26]:
results = functools.reduce(operator.iconcat, results, [])
patents_single_entry['notmeaningful'] = results

C:\Users\fjying\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
patentids = np.unique(patents_multiple_entries['patent_ids'].values)
with Parallel(n_jobs = -1, verbose = 1, prefer = 'threads') as parallel:
    results = parallel(delayed(final_identify_not_meaningful_multiple)(patentid) for patentid in patentids)    

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   30.7s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 4687 out of 4687 | elapsed:  3.0min finished


In [28]:
results = functools.reduce(operator.iconcat, results, [])
patents_multiple_entries['notmeaningful'] = results

C:\Users\fjying\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
patents_meaningful = patents_single_entry.append(patents_multiple_entries)

In [30]:
#reorganize data columns
names = patents_meaningful.head(1).sort_index(axis = 1).columns
columns1 = ['notmeaningful', 'patent_ids','assignor', 'recorded_date', 'last_update_date', 'reel-no', 'frame-no', 'total_number_assignees',
            'name_0','address-1_0', 'address-2_0', 'city_0','country-name_0','postcode_0','state_0']
columns1.extend(list(set(names)-set(columns1)))
patents_meaningful = patents_meaningful[columns1]

In [31]:
patents_meaningful.to_csv("ad20121231-02_notmeaningfultransfer.csv", index = False)